In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pytz
from scipy import interpolate

from pylunar import MoonInfo

%matplotlib inline

In [ ]:
location = ((32, 24, 0), (-111, 0, 0))
#location = ((42, 21, 30), (-71, 3, 35))
obs_datetime = (2020, 10, 1, 5, 0, 0)
ob_obj = datetime(*obs_datetime)
mi = MoonInfo(location[0], location[1])
current_tz = pytz.timezone('US/Arizona')
#current_tz = pytz.timezone('US/Eastern')

In [ ]:
end_date = datetime(2020, 12, 31, 5, 0, 0)
day_inc = 1

In [ ]:
def location_on_sky(t, lat, dec):
    a = np.sin(lat) * np.sin(dec)
    b = np.cos(lat) * np.cos(dec)
    return np.degrees(np.arcsin(a + b * np.cos(t)))

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(121)
latitude = mi.observer.lat
while ob_obj <= end_date:
    utctime = datetime(*ob_obj.timetuple()[:6], tzinfo=pytz.UTC)
    localtime = utctime.astimezone(current_tz)
    local_midnight = current_tz.localize(datetime(*localtime.timetuple()[:3]))
    next_local_midnight = local_midnight + timedelta(hours=24)

    current_time = local_midnight
    az_list = []
    alt_list = []
    ra_list = []
    dec_list = []
    lst_list = []
    while current_time <= next_local_midnight:
        mi.update(current_time.utctimetuple()[:6])
        lst_list.append(mi.observer.sidereal_time())
        az_list.append(mi.azimuth())
        alt_list.append(mi.altitude())
        ra_list.append(mi.ra())
        dec_list.append(mi.dec())
        current_time += timedelta(minutes=30)

    az = np.array(az_list)
    alt = np.array(alt_list)
    ra = np.array(ra_list)
    dec = np.array(dec_list)
    ptime = np.arange(az.size)

    ra_r = np.radians(ra)
    if ra_r[-1] < ra_r[0]:
        diff = ra_r[1:] - ra_r[:-1]
        index = np.where(diff < 0)[0][0] + 1
        scale_ra_r = ra_r[index:] + (2 * np.pi)
        np.put(ra_r, np.arange(index, ra_r.size), scale_ra_r)
    dec_r = np.radians(dec)
    density = 75

    ra_intp = interpolate.interp1d(ptime, ra_r)
    dec_intp = interpolate.interp1d(ptime, dec_r)

    xpp = np.linspace(ptime[0], ptime[-1], density)

    ra_xp = ra_intp(xpp)
    dec_xp = dec_intp(xpp)

    if lst_list[0] < lst_list[-1]:
        lst_start = lst_list[0]
    else:
        lst_start = lst_list[0] - 2 * np.pi

    lst_xp = np.linspace(lst_start, lst_list[-1] + 2 * np.pi, density)
    xp = lst_xp - ra_xp

    ax.plot(ptime, alt, '.', xpp, location_on_sky(xp, latitude, dec_xp), '-')

    ax.axhline(y=0.0, color='k')
    ax.set_title(ob_obj.strftime("%Y/%m/%d"))

    fig.canvas.draw()
    plt.pause(1.0)

    ax.clear()
    ob_obj += timedelta(days=day_inc)